In [65]:
from pylab import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score, train_test_split

# Data Path

In [66]:
data1 = "/Users/Derrick/Desktop/data science new topic/SAT_Results.csv" # data path to data
data2 = "/Users/Derrick/Desktop/data science new topic/nysd.csv"
data3 = "/Users/Derrick/Desktop/data science new topic/NYS_Math_Test_Results_By_Grade_2006-2011_-_School_Level_-_All_Students.csv"
data4 = "/Users/Derrick/Desktop/data science new topic/School_Attendance_and_Enrollment_Statistics_by_District__2010-11_.csv"
data5 = "/Users/Derrick/Desktop/data science new topic/School_Demographics_and_Accountability_Snapshot_2006-2012.csv"
data6 = "/Users/Derrick/Desktop/data science new topic/Graduation_Outcomes_-_Classes_Of_2005-2010_-_School_Level.csv"
data7 = "/Users/Derrick/Desktop/data science new topic/AP__College_Board__2010_School_Level_Results.csv"
data8 = "/Users/Derrick/Desktop/data science new topic/2010-2011_Class_Size_-_School-level_detail.csv"
data9 = "/Users/Derrick/Desktop/data science new topic/masterfile11_d75_final.xlsx"
data10 = "/Users/Derrick/Desktop/data science new topic/masterfile11_gened_final.xlsx"
data11 = "/Users/Derrick/Desktop/data science new topic/Survey Data Dictionary.xls"
data12 = "/Users/Derrick/Desktop/data science new topic/DOE_High_School_Directory_2014-2015.csv"
data13 = "/Users/Derrick/Desktop/data science new topic/School_Progress_Report_2010-2011.csv"
data14 = "/Users/Derrick/Desktop/data science new topic/School_District_Breakdowns.csv"
data15 = "/Users/Derrick/Desktop/data science new topic/attendance.csv"

# Cleaning Data

### SAT result data
 In this data set, it only provided the scores in 3 categories, and we need to add a aveage total score column 
 and we only need two column out of it, DBN and Total Average Score

In [67]:
sat_result = pd.read_csv(data1)
#sat_result.head()

In [68]:
sat_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 6 columns):
DBN                                478 non-null object
SCHOOL NAME                        478 non-null object
Num of SAT Test Takers             478 non-null object
SAT Critical Reading Avg. Score    478 non-null object
SAT Math Avg. Score                478 non-null object
SAT Writing Avg. Score             478 non-null object
dtypes: object(6)
memory usage: 22.5+ KB


In [69]:
new_sat_result = sat_result.drop("Num of SAT Test Takers", 1)
new_sat_result['SAT Math Avg. Score'] = new_sat_result['SAT Math Avg. Score'].convert_objects(convert_numeric = True)
new_sat_result['SAT Critical Reading Avg. Score'] = new_sat_result['SAT Critical Reading Avg. Score'].convert_objects(convert_numeric = True)
new_sat_result['SAT Writing Avg. Score'] = new_sat_result['SAT Writing Avg. Score'].convert_objects(convert_numeric = True)

/Users/Derrick/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app
/Users/Derrick/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()
/Users/Derrick/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [70]:
new_sat_result["Total_Avg"] = new_sat_result.apply(lambda x: x['SAT Math Avg. Score'] + x['SAT Critical Reading Avg. Score'] + x['SAT Writing Avg. Score'], axis=1)

In [71]:
new_sat_result = new_sat_result.rename(columns={"Total_Avg":"SAT_Avg",})
sat_result = new_sat_result[['DBN', 'SAT_Avg']]

### Class Size data
 In this data set, it doesn't provide us the DBN column, and we have to manually made one.
 The DBN column is actually made up of 3 items, CSD, Borough and School Code, and we take that 3 column and made them in to one column

In [72]:
class_size = pd.read_csv(data8)
class_size.head(1)

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN


In [73]:
class_size["DBN"] = class_size.apply(lambda x: "{0:02d}{1}".format(x["CSD"], x["SCHOOL CODE"]), axis=1)
class_size.head(1)

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,DBN
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN,01M015


In [74]:
cols_list = class_size.columns.tolist()
cols_list

['CSD',
 'BOROUGH',
 'SCHOOL CODE',
 'SCHOOL NAME',
 'GRADE ',
 'PROGRAM TYPE',
 'CORE SUBJECT (MS CORE and 9-12 ONLY)',
 'CORE COURSE (MS CORE and 9-12 ONLY)',
 'SERVICE CATEGORY(K-9* ONLY)',
 'NUMBER OF STUDENTS / SEATS FILLED',
 'NUMBER OF SECTIONS',
 'AVERAGE CLASS SIZE',
 'SIZE OF SMALLEST CLASS',
 'SIZE OF LARGEST CLASS',
 'DATA SOURCE',
 'SCHOOLWIDE PUPIL-TEACHER RATIO',
 'DBN']

the DBN column is added at the last column and I want it to show at the first column just like the other data set, and also remove the 3 items we just added in to DBN

In [75]:
cols_swap = cols_list[-1:] + cols_list[:-1]
cols_swap

['DBN',
 'CSD',
 'BOROUGH',
 'SCHOOL CODE',
 'SCHOOL NAME',
 'GRADE ',
 'PROGRAM TYPE',
 'CORE SUBJECT (MS CORE and 9-12 ONLY)',
 'CORE COURSE (MS CORE and 9-12 ONLY)',
 'SERVICE CATEGORY(K-9* ONLY)',
 'NUMBER OF STUDENTS / SEATS FILLED',
 'NUMBER OF SECTIONS',
 'AVERAGE CLASS SIZE',
 'SIZE OF SMALLEST CLASS',
 'SIZE OF LARGEST CLASS',
 'DATA SOURCE',
 'SCHOOLWIDE PUPIL-TEACHER RATIO']

In [76]:
class_size = class_size[cols_swap]
class_size.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27611 entries, 0 to 27610
Data columns (total 17 columns):
DBN                                     27611 non-null object
CSD                                     27611 non-null int64
BOROUGH                                 27611 non-null object
SCHOOL CODE                             27611 non-null object
SCHOOL NAME                             27611 non-null object
GRADE                                   26127 non-null object
PROGRAM TYPE                            26127 non-null object
CORE SUBJECT (MS CORE and 9-12 ONLY)    26127 non-null object
CORE COURSE (MS CORE and 9-12 ONLY)     26127 non-null object
SERVICE CATEGORY(K-9* ONLY)             26127 non-null object
NUMBER OF STUDENTS / SEATS FILLED       26127 non-null float64
NUMBER OF SECTIONS                      26127 non-null float64
AVERAGE CLASS SIZE                      26127 non-null float64
SIZE OF SMALLEST CLASS                  26127 non-null float64
SIZE OF LARGEST CLAS

### Attendence

In [77]:
attd_2010 = pd.read_csv(data4)
attd_2010.head()

,District,YTD % Attendance (Avg),YTD Enrollment(Avg)
0,DISTRICT 01,91.18%,12367
1,DISTRICT 02,89.01%,60823
2,DISTRICT 03,89.28%,21962
3,DISTRICT 04,91.13%,14252
4,DISTRICT 05,89.08%,13170


In [78]:
new_attendance = attd_2010
new_attendance['District'] = attd_2010.apply(lambda x: x['District'].replace('DISTRICT ', ''), axis=1)
new_attendance = new_attendance.drop([33, 34])
new_attendance['District'] = new_attendance.apply(lambda x: x['District'].replace('SPECIAL ED ', ''), axis=1)
new_attendance['YTD % Attendance (Avg)'] = new_attendance.apply(lambda x: x['YTD % Attendance (Avg)'].replace('%', ''), axis=1)
new_attendance = new_attendance.rename(columns={'YTD % Attendance (Avg)':'Avg_Attendance', 'YTD Enrollment(Avg)':'Avg_enrollment'})

In [79]:
new_attendance = new_attendance.rename(columns={'Avg_Attendance':"Avg_atendance" })
new_attendance['Avg_atendance'] = new_attendance['Avg_atendance'].convert_objects(convert_numeric = True)

/Users/Derrick/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [80]:
attd_2010 = new_attendance

### School Progress Report (school grade)

In [81]:
progress = pd.read_csv(data13)
progress.head(1)

,DBN,DISTRICT,SCHOOL,PRINCIPAL,PROGRESS REPORT TYPE,SCHOOL LEVEL*,PEER INDEX*,2010-2011 OVERALL GRADE,2010-2011 OVERALL SCORE,2010-2011 ENVIRONMENT CATEGORY SCORE,2010-2011 ENVIRONMENT GRADE,2010-2011 PERFORMANCE CATEGORY SCORE,2010-2011 PERFORMANCE GRADE,2010-2011 PROGRESS CATEGORY SCORE,2010-2011 PROGRESS GRADE,2010-2011 ADDITIONAL CREDIT,2009-10 PROGRESS REPORT GRADE
0,01M015,1,P.S. 015 Roberto Clemente,Irene Sanchez,EMS,Elementary,63.61,C,27.0,6.4,B,2.1,D,18.0,C,0.5,C


In [82]:
school_grade = progress[progress["SCHOOL LEVEL*"] == "High School"]

In [83]:
school_grade = school_grade.drop(["DISTRICT", "SCHOOL", "PRINCIPAL", "PROGRESS REPORT TYPE", "SCHOOL LEVEL*"], axis=1)

In [84]:
school_grade = school_grade.rename(columns={'\ufeffDBN':'DBN'})

### High School Directory
For drawing map

In [85]:
hs_dir = pd.read_csv(data12)
hs_dir.head(1)

,dbn,school_name,boro,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,priority02,priority03,priority04,priority05,priority06,priority07,priority08,priority09,priority10,Location 1
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9,12,NaN,NaN,...,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67..."


Rename column "dbn" to "DBN" by making another copy of the column and delete the "dbn" 

In [86]:
hs_dir["DBN"] = hs_dir["dbn"]

In [87]:
hs_dir = hs_dir.drop(hs_dir.columns[[0]], axis=1) #drop the old "dbn" column

In [88]:
cols_list2 = hs_dir.columns.tolist()

In [89]:
cols2_swap2 = cols_list2[-1:] + cols_list2[:-1] #swap the DBN column into the first column
cols2_swap2[:3]

['DBN', 'school_name', 'boro']

### Addition data cleaning for our class size data set
select only grade 9-12, and only general education and then groupby DBN column

In [90]:
new_class_size = class_size
class_size = class_size[class_size["GRADE "] == "09-12"]
class_size = class_size[class_size["PROGRAM TYPE"] == "GEN ED"]
class_size = class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace=True)
class_size = new_class_size

### Math Test Result

In [97]:
math_test_result = pd.read_csv(data3)
math_test_result.head(1)

,DBN,Grade,Year,Category,Number Tested,Mean Scale Score,Level 1 #,Level 1 %,Level 2 #,Level 2 %,Level 3 #,Level 3 %,Level 4 #,Level 4 %,Level 3+4 #,Level 3+4 %
0,01M015,3,2006,All Students,39,667,2,5.1%,11,28.2%,20,51.3%,6,15.4%,26,66.7%


In [98]:
math_test_result = math_test_result[math_test_result["Year"] == 2011]
math_test_result = math_test_result[math_test_result["Grade"] == "8"]

# Merge Data

for map drawing purpose

In [99]:
data = {'sat_result': sat_result, 
        #'class_size': class_size, 
        'hs_dir': hs_dir, 
        'math_test_result': math_test_result,
        #'demographic': demographic,
        #'graduation_2010': graduation_2010,
        #'ap_2010': ap_2010,
        #'attd_2010': attd_2010,
        'progress': progress}

In [102]:
flat_data_names = [k for k,v in data.items()]
flat_data = [data[k] for k in flat_data_names]
final = flat_data[0]
for i, f in enumerate(flat_data[1:]):
    name = flat_data_names[i+1]
    #print(name)
    #print(len(f['DBN']) - len(f['DBN'].unique()))
    join_type = "inner"
    if name in ["sat_result"]:
        join_type = "outer"
    if name not in ["math_test_result"]:
        final = final.merge(f, on="DBN", how=join_type)

final.shape

(440, 75)

for visualizing data and machine learning

In [ ]:
len()